# Wordle

This notebook was created by Idrees Hussain.

# The Programme

## Setup

Here, we simply include some imports for the later, as well as download a list of valid 5-letter words. 

I have surrounded the download attempt in a try-catch statement. This means that if the Internet Connection is not functioning correctly for whatever reason, the program will inform the user before terminating. This prevents an error from occuring and forcing the program into an unexpected state.

In [212]:
from dataclasses import dataclass   # for defining dataclasses
from enum import Enum               # for defining enumerations
import typing                       # for type hinting
import random                       # for selecting a random answer
from typing import List             # for defining custom typed lists
from urllib.request import urlopen  # to load web data

# grab a list of words

url = 'https://raw.githubusercontent.com/tabatkins/wordle-list/main/words'
try:
    WORDS = [word.rstrip().decode('UTF-8').upper() for word in urlopen(url).readlines()]
    setWords = set(WORDS)
except URLError as e:
    print("Internet Connection Error: {0}".format(e))
    print("Exiting...")
    quit()

## Data Types and Contants

Next, we can define some simple data types, and constants that we will need to use, based on the above analysis. Here we define:

* The length of a valid word.
* The maximum number of guesses.
* A `Word` type as an alias of `str`
* An enumeration of the three types of clue (i.e. colours)
* An enumeration for the game state, either in play, won, or lost.

In [213]:
# constants

WORD_LENGTH = 5
MAX_GUESSES = 6
Word = str


# type enumerating the three possible clue colours

class Clue(Enum):
    GREEN = 'GREEN'
    YELLOW = 'YELLOW'
    GREY = 'GREY'


class Gamestate(Enum):
    WON = 'WON'
    LOST = 'LOST'
    PLAYING = 'PLAYING'

## Guess

We can now introduce a `Guess` type, which represents a guess after it is processed by the game (i.e. one line of the output). We use the `dataclass' decorator and properties to simplify definition. This includes the word that was guessed, and the clues.

1. There are two invariants to add to the `setter` functions; what are these? Complete the assertions and include an appropriate message for when they fail. These should restrict the values that `word` and `clues` can take. 
2. Create some test values for `Guess` to demonstrate the invariants, i.e. `g1 = Guess("HELLO", [Clue.YELLOW, ...])`.

In [214]:
@dataclass
class Guess:
    """
    A class to represent a guess in Wordle, which is a Word 
    plus a clue for each letter.
    """
    word: Word
    clues: List[Clue]

    """
    Getter and Setter for word
    """

    @property
    def word(self) -> Word:
        return self._word

    @word.setter
    def word(self, word: Word):
        assert isinstance(word, Word), "Word must be of type word"

        self._word = word

    """
    Getter and Setter for Clue
    """

    @property
    def clues(self) -> List[Clue]:
        return self._clues

    @clues.setter
    def clues(self, clues: Clue):
        assert isinstance(clues, list)

        self._clues = clues


    def __repr__(self):
        """
        Custom representation for pretty printing.
        """
        cluestr = [str(self.word[i]) + ": " + \
            self.clues[i].name for i in range(WORD_LENGTH)]
        return f"{self.word}: {cluestr}"    
           


# Auxiliary Functions

There are multiple ways to specify the game. The suggestion here is to have a main class `Game` that includes the state and top-level functions including `make_guess`. To help define those, we define two auxiliary functions:

* `check_letter`: Compute clue given a character, its position, and word
* `check_guess`: Compute clues given a word and the answer

## check_letter 

This function computes the clue (colour) for a given letter, given a word and the index of the letter in the word. We will start with a naïve implementation:

* If it's in the right place, return green (i.e. `Clue.GREEN`)
* If it’s not there, return grey
* Otherwise, return yellow.

Complete the function in the following way:

3. What pre-conditions should be included? These should restrict the parameters. Add these and an appropriate message. 
4. Complete the implementation based on the sketch above.
5. Include some test cases to check your implementation, e.g. `print(check_letter("S", 0, "STOUT"))`.

In [215]:
def check_letter(letter: str, index: int, word: Word) -> Clue:
    """
    Given a letter and an index, computes the colour of the blue
    based on the word.
    """
    assert isinstance(index, int) and \
           isinstance(letter, str) and \
           isinstance(word, Word) and \
           0 <= index <= WORD_LENGTH and \
           len(letter) == 1 and \
           len(word) == WORD_LENGTH, "pre-check_letter failed"

    if word[index: index + 1] == letter:
        return Clue.GREEN

    if letter in word:
        return Clue.YELLOW

    return Clue.GREY



## Check Guess

Given a word and the correct answer, this function computes the list of clues. This can be achieved by declaring a local variable, and for each letter in the guess, use `check_letter`; we can use `enumerate(guess)` to get the index.

6. What pre-conditions should be included? These should restrict the parameters. Add these and an appropriate message. 
7. Complete the implementation based on the sketch above.
8. Include some test cases to check your implementation, e.g. `print(check_guess("STAND", "STOUT"))`.

In [216]:
def check_guess(word: Word, guess: Word) -> List[Clue]:
    """
    Given the answer and a guess, compute the list of 
    clues corresponding to each letter.
    """
    assert isinstance(word, Word) and \
           isinstance(guess, Word) and \
           len(guess) == WORD_LENGTH and \
           len(word) == WORD_LENGTH, "pre-check_guess failed"

    clues = []
    for index, letter in enumerate(guess):
        clues.append(check_letter(guess[index: index + 1], index, word))

    return clues


# Game State and Top-level Functionality

## Game

In [217]:
class Game:
    answer: Word
    guesses: List[Guess]
    gstate: Gamestate

    def __init__(self, answer="STOUT"):
        """
        Constructor for game.
        """
        self.answer = answer
        self.guesses = []
        self.gstate = Gamestate.PLAYING

    def make_guess(self, word: Word):
        """
        Make a guess at the wordle.
        """
        assert self.gstate == Gamestate.PLAYING and \
               isinstance(word, Word) and \
               len(word) == WORD_LENGTH and \
               word.upper() in setWords, "make-guess assertion failed"

        word = word.upper()
        guess = Guess(word, check_guess(self.answer, word))
        self.guesses.append(guess)

        if not (Clue.YELLOW in guess.clues or Clue.GREY in guess.clues):
            self.gstate = Gamestate.WON

        if len(self.guesses) == 6:
            self.gstate = Gamestate.LOST

    def print_state(self):
        """
        Prints a message to the user based on the current state of the game.
        """
        for guess in self.guesses: print(guess)
        if self.gstate == Gamestate.WON: print(f"You won! You took {len(self.guesses)} guesses.")
        elif self.gstate == Gamestate.LOST: print(f"You lost! The answer was: {self.answer}.")
        else: print(f"Guess the wordle, you have {MAX_GUESSES - len(self.guesses)} guesses remaining.")

    def game_over(self) -> bool:
        """
        Yields true if the game is over (won or lost), false otherwise.
        """
        return self.gstate == Gamestate.WON or self.gstate == Gamestate.LOST 

    def reset(self):
        """
        Reset the game by picking a new word, clearing the guess, and
        setting the state back to playing.
        """
        # pre-condition
        assert self.game_over(), "Cannot reset, game in play"
        self.answer = random.choice(WORDS)
        self.guesses = []
        self.gstate = Gamestate.PLAYING
    
    def print_hint(self):
        lst = []
        for letter in self.answer:
            lst.append(letter)

        hint = Hint(random.choice(lst))
        print("HINT: The letter {0} is the {1} letter in the word.".format(
            hint, numbers_to_ordinal(lst.index(str(hint)))))


# Extended Tasks

## Hint

In [218]:
@dataclass
class Hint:
    # Hint is defined as a single alphabetic uppercase character
    def __init__(self, hint):
        
        assert isinstance(hint, str) and \
            len(hint) == 1 and \
            hint.isalpha() and hint.isupper(), "Hint pre-condition failed"

        # Sets hint to inputted value
        self.hint = hint

    def __str__(self):
        return self.hint

def numbers_to_ordinal(number: int) -> str:
    assert isinstance(number, int) and 0 <= number <= 4
    match number:
        case 0:
            return "1st"
        case 1:
            return "2nd"
        case 2:
            return "3rd"
        case 3:
            return "4th"
        case 4:
            return "5th"
        case _:
            return "Error Occurred"
        
def playgame():
    game = Game()

    game.gstate = Gamestate.WON

    # Reset game to reset word from default

    game.reset()
    while game.gstate == Gamestate.PLAYING:
        game.print_state()
        Input = "TEST"

        while Input.upper() not in setWords:
            if Input.upper() == "HINT":
                game.print_state()
                game.print_hint()
            Input = input("Please enter a valid word\n")

        game.make_guess(Input)

    game.print_state()
    pass

## Improved check_guess implementation

In [219]:
def check_green(letter: str, index: int, word: Word, char_count: dict) -> [Clue, dict]:
    if word[index: index + 1] == letter:
        char_count[letter] -= 1
        return Clue.GREEN, char_count

    return Clue.GREY, char_count


def check_yellow(letter: str, word: Word, char_count: dict) -> [Clue, dict]:
    if letter in word and char_count[letter] != 0:
        char_count[letter] -= 1
        return Clue.YELLOW, char_count

    return None, char_count


def check_guess(word: Word, guess: Word) -> List[Clue]:
    """
    Given the answer and a guess compute the list of clues corresponding to each letter
    """

    assert isinstance(word, Word) and \
           isinstance(guess, Word) and \
           len(guess) == WORD_LENGTH and \
           len(word) == WORD_LENGTH, "pre-check_guess failed"

    # Create a dictionary with a char count
    char_count = {}
    for char in word:
        if char in char_count.keys():
            char_count[char] += 1
        else:
            char_count[char] = 1

    clues = [Clue.GREY for i in range(WORD_LENGTH)]

    # Check Green

    for index, letter in enumerate(guess):
        clues[index], char_count = check_green(letter, index, word, char_count)

    # Check Yellow

    for index, letter in enumerate(guess):
        temp, char_count = check_yellow(letter, word, char_count)
        if temp is not None:
            clues[index] = temp

    return clues

## Hard Mode

In [220]:
def playHardMode():
    game = Game()
    game.gstate = Gamestate.WON

    # Reset game to reset word from default

    game.reset()

    while game.gstate == Gamestate.PLAYING:
        game.print_state()
        Input = "TEST"
        while Input.upper() not in setWords:
            # HARD MODE CODE
            #
            Input = input("Please enter a valid word\n")
            Input = Input.upper()

            if Input == "HINT":
                game.print_state()
                game.print_hint()

            if Input in setWords:
                if len(game.guesses) != 0:
                    index = -1
                    for clue in game.guesses[len(game.guesses) - 1].clues:
                        index += 1
                        if clue == Clue.GREEN and Input[index] != game.guesses[len(game.guesses) - 1].word[index]:
                            Input = "TEST"
                            break

                        if clue == Clue.YELLOW and game.guesses[len(game.guesses) - 1].word[index] not in Input:
                            Input = "TEST"
                            break

        game.make_guess(Input)

    game.print_state()
    pass

## Tests

In [221]:
passed_tests = 0
tests_attempted = 0

try:
    assert (check_letter("S", 0, "STOUT")) == Clue.GREEN
    print("TEST PASSED")
    tests_attempted += 1
    passed_tests += 1
except AssertionError as error:
    print("TEST FAILED")
    tests_attempted += 1

try:
    assert (check_letter("S", 3, "STOUT")) == Clue.YELLOW 
    print("TEST PASSED")
    tests_attempted += 1
    passed_tests += 1
except AssertionError as error:
    print("TEST FAILED")
    tests_attempted += 1

try:
    assert (check_letter("Z", 0, "STOUT")) == Clue.GREY
    print("TEST PASSED")
    passed_tests += 1
    tests_attempted += 1
except AssertionError as error:
    print("TEST FAILED")
    tests_attempted += 1

try:
    assert check_guess("STAND", "STOUT") == [Clue.GREEN, Clue.GREEN, Clue.GREY, Clue.GREY, Clue.GREY]
    print("TEST PASSED")
    tests_attempted += 1
    passed_tests += 1
except AssertionError as error:
    print("TEST FAILED")
    tests_attempted += 1

try:
    assert check_guess("LOOPS", "IGLOO") == [Clue.GREY, Clue.GREY, Clue.YELLOW, Clue.YELLOW, Clue.YELLOW]
    print("TEST PASSED")
    passed_tests += 1
    tests_attempted += 1
except AssertionError as error:
    print("TEST FAILED")
    tests_attempted += 1

try:
    hint1 = Hint('A')
    print("TEST PASSED")
    passed_tests += 1
    tests_attempted += 1
except AssertionError as error:
    print("TEST FAILED")
    tests_attempted += 1

try:
    hint2 = Hint('AB')
    print("TEST FAILED")
    tests_attempted += 1
except AssertionError as error:
    print("TEST PASSED")
    passed_tests += 1
    tests_attempted += 1

try:
    hint2 = Hint('a')
    print("TEST FAILED")
    tests_attempted += 1
except AssertionError as error:
    print("TEST PASSED")
    passed_tests += 1
    tests_attempted += 1

print("Successfully passed {0}/{1} tests".format(passed_tests, tests_attempted))

TEST PASSED
TEST PASSED
TEST PASSED
TEST PASSED
TEST PASSED
TEST PASSED
TEST PASSED
TEST PASSED
Successfully passed 8/8 tests


# Further Expansion
## Game Menu

In [222]:
import time  # For adding delays in code

def menu():
    """
    Function to print and navigate menu
    """
    while True:
        # Print Menu Options
        print("""
        1) Play Game
        2) Hard Mode
        3) Tests
        4) Quit
        """)
        try:
            # Accept user input to navigate menu
            int_input = int(input("Enter your choice: "))

            match int_input:
                case 1:
                    # Regular Game Mode
                    playgame()
                case 2:
                    # Hard Mode
                    playHardMode()
                case 3:
                    # Test Scenarios
                    test()
                case 4:
                    # Quit
                    exit()
                    quit()
                case _:
                    # If other numbers are entered
                    print("Invalid Entry")
                    time.sleep(1)
        except ValueError:
            # If input by user is not a number
            print("Invalid Entry")
            time.sleep(1)


def main():
    menu()

## Visualisation

### GUI Prototype